## Imports

In [41]:
import numpy as np
import open3d as o3d
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

## Plotting

In [42]:
# FUNCTION: 
#    This function plots a point cloud
#
# INPUT:
#   - cloud (open3d.geometry.PointCloud): original point cloud on which the operation is performed 
#   - epsilon (float): epsilon parameter for the dbscan algorithm (optional)
#   - min_points (int): min_points parameter for the dbscan algorithm (optional)
#   - correct_rotation (boolean): rotates point cloud to ideal position for this project, overwrites mirrorX/mirrorY/mirrorZ
#   - shaded (boolean): applies a simple shading for a 3D effect if true
#   - mirrorX (boolean): mirrors cloud along the X-axis
#   - mirrorY (boolean): mirrors cloud along the Y-axis
#   - mirrorZ (boolean): mirrors cloud along the Z-axis

def plotCloud(cloud, correct_rotation = False, shaded = False, mirrorX = False, mirrorY = False, mirrorZ = False):
    if(correct_rotation):
        mirrorX = False
        mirrorY = True
        mirrorZ = True
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')
    
    colors = cloud.colors
    if(shaded):
        cloud.estimate_normals()
        cloud.normalize_normals()
        normals = np.absolute(np.asarray(cloud.normals))
        colors = []
        for n,c in zip(normals,cloud.colors):
            colors.append(c * (np.dot(n,[0,0,1])*0.8+0.2))
        
        
    points = np.asarray(cloud.points)
            
    ax.scatter(points[:,0]*-(mirrorX*2-1),points[:,1]*-(mirrorY*2-1),points[:,2]*-(mirrorZ*2-1), c = colors, marker = ".", alpha = 0.2)
    plt.show()
    

## Paint Clusters

In [43]:
cluster_colors = [[0, 0, 1],[0, 1, 1], [0, 1, 0], [1, 1, 0], [1, 0, 1], [1, 0, 0], [1, 0.5, 0], [1, 0, 0.5], 
                  [0.5, 1, 0], [0, 1, 0.5], [0.5, 0, 1], [0, 0.5, 1], [1, 0.5, 0.5], [0.5, 1, 0.5], [0.5, 0.5, 1]]

# FUNCTION:
#   This function applies the dbscan clustering algorithm to a point cloud and paints the clusters accordingly
#
# INPUT:
#   - cloud (open3d.geometry.PointCloud): original point cloud on which the operation is performed 
#   - epsilon (float): epsilon parameter for the dbscan algorithm (optional)
#   - min_points (int): min_points parameter for the dbscan algorithm (optional)
#   - colors (float array, shape: [n,3]): list of colors in which to paint the clusters

def paintClusters(cloud, epsilon = 0.3, min_points = 3, colors = cluster_colors):
    clustering = np.asarray(cloud.cluster_dbscan(epsilon, min_points))
    result = []
    for c in clustering:
        if c>=len(colors):
            result.append([0.5,0.5,0.5])
        else:
            result.append(colors[c])
    cloud.colors = o3d.utility.Vector3dVector(result)

## Outlier Removal

In [48]:
# FUNCTION:
#   This function removes outliers in a point cloud by applying the dbscan clustering algorithm and removing small clusters
#
# INPUT:
#   - cloud (open3d.geometry.PointCloud): original point cloud on which the operation is performed 
#   - threshold (int/float): size below wich clusters are removed. If theshold >= 1, then the absolut value is used,
#                            if threshold < 1 then the threshold is calculated as percentage of the biggest cluster
#   - epsilon (float): epsilon parameter for the dbscan algorithm (optional)
#   - min_points (int): min_points parameter for the dbscan algorithm (optional)

def removeOutliers(cloud, threshold = 1000, epsilon = 0.3, min_points = 3):
    clustering = np.asarray(cloud.cluster_dbscan(epsilon, min_points))
    
    #count cluster
    number_of_cl = max(np.asarray(clustering))
    unique, c = np.unique(np.asarray(clustering), return_counts=True)
    
    #if threshold is given as percentage, calculate real threshold
    if threshold < 1:
        threshold = max(c) * threshold
    
    
    counts = dict(zip(unique,c))
    final_cl = [0]*len(clustering)
    
    points_old = np.asarray(cloud.points)
    points_new = []
    for i in range(len(clustering)):
        if counts[clustering[i]] > threshold:
            #final_cl[i] = 1
            points_new.append(points_old[i])
        #else:
            #final_cl[i] = 0
        #print(i,db.labels_[i],counts[db.labels_[i]],final_cl[i])
            
    #points = np.asarray(cloud.points)[final_cl]
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_new)
    pcd.paint_uniform_color([0.5, 0.5, 0.5])
    return pcd

In [45]:
print("Clustering and Plotting loaded")

Clustering and Plotting loaded


## Testing Area